In [3]:
# sklearn imports
from sklearn.neighbors import NearestNeighbors, KNeighborsTransformer

# data science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# imodels imports
from imodels.imodels import get_clean_dataset


ModuleNotFoundError: No module named 'imodels'